In [1]:
%pylab inline
import os, os.path
from galpy.actionAngle import actionAngleIsochroneApprox, \
    actionAngleTorus
from galpy.potential import LogarithmicHaloPotential
from galpy.orbit import Orbit
from galpy.util import bovy_coords, bovy_conversion

Populating the interactive namespace from numpy and matplotlib


# Some checks on the accuracy of Jo's action-angle calculations

In [2]:
maxn= 3

Potential etc.

In [3]:
lp= LogarithmicHaloPotential(normalize=1.,q=0.9)
R0, V0= 8., 220.
def convert_to_cylindrical(xv):
    R,phi,Z= bovy_coords.rect_to_cyl(xv[:,0],xv[:,1],xv[:,2])
    vR,vT,vZ= bovy_coords.rect_to_cyl_vec(xv[:,3],xv[:,4],xv[:,5],R,phi,Z,cyl=True)
    out= numpy.empty_like(xv)
    # Preferred galpy arrangement of cylindrical coordinates
    out[:,0]= R
    out[:,1]= vR
    out[:,2]= vT
    out[:,3]= Z
    out[:,4]= vZ
    out[:,5]= phi
    return out

In [4]:
xv_prog_init= numpy.array([30.,0.,0.,0., 105.74895, 105.74895])
RvR_prog_init= convert_to_cylindrical(xv_prog_init[:,numpy.newaxis].T)[0,:]
prog_init= Orbit([RvR_prog_init[0]/R0,RvR_prog_init[1]/V0,RvR_prog_init[2]/V0,
                  RvR_prog_init[3]/R0,RvR_prog_init[4]/V0,RvR_prog_init[5]],ro=R0,vo=V0)
times= numpy.linspace(0.,10.88/bovy_conversion.time_in_Gyr(V0,R0),1001)
prog_init.integrate(times,lp)
xv_unp_peri_prog= [prog_init.x(times[-1])[0],prog_init.y(times[-1])[0],prog_init.z(times[-1]),
              prog_init.vx(times[-1])[0],prog_init.vy(times[-1])[0],prog_init.vz(times[-1])]
RvR_unp_peri_prog= convert_to_cylindrical(numpy.array(xv_unp_peri_prog)[:,numpy.newaxis].T)[0,:]
prog_unp_peri= Orbit([RvR_unp_peri_prog[0]/R0,RvR_unp_peri_prog[1]/V0,RvR_unp_peri_prog[2]/V0,
                      RvR_unp_peri_prog[3]/R0+0.0,RvR_unp_peri_prog[4]/V0,RvR_unp_peri_prog[5]]) # Normalized coordinates

We calculate the actions using actionAngleIsochroneApprox, then calculate the phase-space position using actionAngleTorus and then again the actions, frequencies, and angles using actionAngleIsochroneApprox

In [5]:
aAIA= actionAngleIsochroneApprox(pot=lp,b=0.8)
aAT= actionAngleTorus(pot=lp,tol=10.**-3.)

In [6]:
w_start= aAIA.actionsFreqsAngles(prog_unp_peri,maxn=maxn)

In [7]:
xvFreqs_torus= aAT.xvFreqs(w_start[0],w_start[1],w_start[2],
                          w_start[6],w_start[7],w_start[8])

In [8]:
w_final= aAIA.actionsFreqsAngles(*xvFreqs_torus[:6],maxn=maxn)

Actions

In [9]:
print (w_final[0]-w_start[0])/w_start[0]
print (w_final[1]-w_start[1])/w_start[1]
print (w_final[2]-w_start[2])/w_start[2]

[ 0.00962444]
[  1.23184361e-16]
[-0.00643492]


Frequencies

In [10]:
print (w_final[3]-w_start[3])/w_start[3]
print (w_final[4]-w_start[4])/w_start[4]
print (w_final[5]-w_start[5])/w_start[5]

[ 0.00017314]
[  8.92091775e-05]
[ 0.0003473]


Angles

In [11]:
print w_final[6]-w_start[6]
print w_final[7]-w_start[7]
print w_final[8]-w_start[8]

[ -6.41139326e-05]
[ 0.00042028]
[-0.00043099]


## Same for the gap location

In [12]:
xv_gap= numpy.loadtxt(os.path.join(os.getenv('DATADIR'),'streamgap-aa','nonzeroJz','gap_location_2nd_peri.txt'))[:6]
RvR_gap= convert_to_cylindrical(xv_gap[:,numpy.newaxis].T)[0,:]
gap_orb= Orbit([RvR_gap[0]/R0,RvR_gap[1]/V0,RvR_gap[2]/V0,
                RvR_gap[3]/R0,RvR_gap[4]/V0,RvR_gap[5]])

In [13]:
w_start= aAIA.actionsFreqsAngles(gap_orb,maxn=maxn)

In [14]:
xvFreqs_torus= aAT.xvFreqs(w_start[0],w_start[1],w_start[2],
                          w_start[6],w_start[7],w_start[8])

In [15]:
w_final= aAIA.actionsFreqsAngles(*xvFreqs_torus[:6],maxn=maxn)

Actions

In [16]:
print (w_final[0]-w_start[0])/w_start[0]
print (w_final[1]-w_start[1])/w_start[1]
print (w_final[2]-w_start[2])/w_start[2]

[ 0.0043883]
[  1.21471068e-16]
[-0.00211472]


Frequencies

In [17]:
print (w_final[3]-w_start[3])/w_start[3]
print (w_final[4]-w_start[4])/w_start[4]
print (w_final[5]-w_start[5])/w_start[5]

[ -8.97784663e-05]
[-0.00013358]
[ -3.39723704e-05]


Angles

In [18]:
print w_final[6]-w_start[6]
print w_final[7]-w_start[7]
print w_final[8]-w_start[8]

[-0.00056908]
[ 0.00092115]
[ 0.00073651]
